In [ ]:
import os

from hubspot import HubSpot
import pandas as pd


ACCESS_TOKEN = os.environ["HUBSPOT_ACCESS_TOKEN"]

hubspot = HubSpot(access_token=ACCESS_TOKEN)

In [ ]:
company_props = hubspot.crm.properties.core_api.get_all(object_type="companies", archived=False)

In [ ]:
# read results into DataFrame
company_props_df = pd.json_normalize(company_props.to_dict(), "results")

# assign dtypes for columns used later
dtypes = {
    "hubspot_defined": "bool",
}
for field, dtype in dtypes.items():
    company_props_df[field] = company_props_df[field].astype(dtype)

In [ ]:
hubspot_props = company_props_df[company_props_df["hubspot_defined"]]
hubspot_props.to_json("data/company_props_hubspot.json", orient="records", indent=2)

In [ ]:
custom_props = company_props_df[~company_props_df["hubspot_defined"]]
custom_props.to_json("data/company_props_custom.json", orient="records", indent=2)

In [ ]:
companies = hubspot.crm.companies.get_all(properties=["company_type", "domain", "name"], associations=["calls", "emails", "meetings", "notes", "tasks"])
companies = [c.to_dict() for c in companies]

In [ ]:
companies_df = pd.json_normalize(companies)
companies_df.to_json("data/companies.json", orient="records", indent=2)

In [ ]:
# look at the unique company types defined
companies_df["properties.company_type"].unique().tofile("data/company_types.txt", sep=os.linesep)

In [ ]:
vendor_companies_df = companies_df[companies_df["properties.company_type"] == "Vendor"]
vendor_companies_df.to_json("data/vendors.json", orient="records", indent=2)